In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
crimes = pd.read_csv('data\crimes.csv')
data = crimes[["dispatch_date", "dispatch_time","location_block", "text_general_code", "point_x", "point_y"]]

In [3]:
data.head()

,dispatch_date,dispatch_time,location_block,text_general_code,point_x,point_y
0,2023-10-04,22:58:00,1300 BLOCK N ALLISON ST,Thefts,-75.232271,39.972757
1,2023-10-04,22:42:00,200 BLOCK W CHELTEN AV,DRIVING UNDER THE INFLUENCE,-75.178605,40.030809
2,2023-10-04,22:25:00,600 BLOCK N 52ND ST,All Other Offenses,-75.226049,39.971074
3,2023-10-04,22:23:00,3400 BLOCK ARBOR ST,All Other Offenses,-75.121491,40.001767
4,2023-10-04,22:17:00,5900 BLOCK N BROAD ST,Other Assaults,-75.143587,40.044345


# Feature Preprocessing

Dissect Date and Time into a set of categorical features

In [4]:
data['dispatch_date'] = pd.to_datetime(data['dispatch_date'])
data['dispatch_time'] = pd.to_datetime(data['dispatch_time'], format='%H:%M:%S').dt.time

# Combine date and time into a single DateTime column
data['DateTime'] = pd.to_datetime(data['dispatch_date'].astype(str) + ' ' + data['dispatch_time'].astype(str))

# Extracting features
data['Year'] = data['DateTime'].dt.year
data['Month'] = data['DateTime'].dt.month
data['Day'] = data['DateTime'].dt.day
data['Hour'] = data['DateTime'].dt.hour
data['Minute'] = data['DateTime'].dt.minute
data['Weekday'] = data['DateTime'].dt.weekday  # Monday=0, Sunday=6
data['IsWeekend'] = data['Weekday'].apply(lambda x: 1 if x >= 5 else 0)

C:\Users\Alex Zavalny\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [5]:
data.head()

,dispatch_date,dispatch_time,location_block,text_general_code,point_x,point_y,DateTime,Year,Month,Day,Hour,Minute,Weekday,IsWeekend
0,2023-10-04,22:58:00,1300 BLOCK N ALLISON ST,Thefts,-75.232271,39.972757,2023-10-04 22:58:00,2023,10,4,22,58,2,0
1,2023-10-04,22:42:00,200 BLOCK W CHELTEN AV,DRIVING UNDER THE INFLUENCE,-75.178605,40.030809,2023-10-04 22:42:00,2023,10,4,22,42,2,0
2,2023-10-04,22:25:00,600 BLOCK N 52ND ST,All Other Offenses,-75.226049,39.971074,2023-10-04 22:25:00,2023,10,4,22,25,2,0
3,2023-10-04,22:23:00,3400 BLOCK ARBOR ST,All Other Offenses,-75.121491,40.001767,2023-10-04 22:23:00,2023,10,4,22,23,2,0
4,2023-10-04,22:17:00,5900 BLOCK N BROAD ST,Other Assaults,-75.143587,40.044345,2023-10-04 22:17:00,2023,10,4,22,17,2,0


Extracted street names and block numbers

In [6]:
locations=[]
street_name = []
for location in data["location_block"]:
    if type(location) == float:
        splitted = ""
    else:
        splitted = location.split( " BLOCK ")
    if len(splitted) == 2:
        locations.append(splitted[0])
        street_name.append(splitted[1])
    else:
        locations.append(0)
        street_name.append(splitted)

In [7]:
street_name_proc = [s[0] for s in street_name if type(s) == list] + [s for s in street_name if type(s) == str]

data["location"] = pd.Series(locations).values
data["street_name"] = pd.Series(street_name_proc).values
data = data.drop("location_block", axis=1)

C:\Users\Alex Zavalny\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [8]:
data.head()

,dispatch_date,dispatch_time,text_general_code,point_x,point_y,DateTime,Year,Month,Day,Hour,Minute,Weekday,IsWeekend,location,street_name
0,2023-10-04,22:58:00,Thefts,-75.232271,39.972757,2023-10-04 22:58:00,2023,10,4,22,58,2,0,1300,Main Toll Plaza Ln 84
1,2023-10-04,22:42:00,DRIVING UNDER THE INFLUENCE,-75.178605,40.030809,2023-10-04 22:42:00,2023,10,4,22,42,2,0,200,FRANKFORD AV & ROBBINS ST
2,2023-10-04,22:25:00,All Other Offenses,-75.226049,39.971074,2023-10-04 22:25:00,2023,10,4,22,25,2,0,600,N 25TH ST & W CAMBRIA ST
3,2023-10-04,22:23:00,All Other Offenses,-75.121491,40.001767,2023-10-04 22:23:00,2023,10,4,22,23,2,0,3400,Rt1 S&b At Wissahickon Ave
4,2023-10-04,22:17:00,Other Assaults,-75.143587,40.044345,2023-10-04 22:17:00,2023,10,4,22,17,2,0,5900,N 3RD ST & W ALLEGHENY AV


One Hot Encoding Crime Type, since there is no inherent order in the crime types

In [9]:
data.rename(columns={'text_general_code':'crime_type'}, inplace=True)
data["crime_type"].value_counts()

All Other Offenses                         567720
Other Assaults                             435605
Thefts                                     431659
Vandalism/Criminal Mischief                297890
Theft from Vehicle                         259640
Narcotic / Drug Law Violations             180360
Fraud                                      176292
Burglary Residential                       122241
Aggravated Assault No Firearm              103848
Motor Vehicle Theft                         91499
Robbery No Firearm                          71680
DRIVING UNDER THE INFLUENCE                 66617
Robbery Firearm                             56052
Aggravated Assault Firearm                  48277
Disorderly Conduct                          44115
Weapon Violations                           38339
Burglary Non-Residential                    35676
Other Sex Offenses (Not Commercialized)     21524
Rape                                        17617
Prostitution and Commercialized Vice        15934


In [10]:
# combining Homicide categories together
data["crime_type"] = data["crime_type"].apply(lambda x: "Homicides" if isinstance(x, str) and "Homicide" in x else x)
data["crime_type"].value_counts()

All Other Offenses                         567720
Other Assaults                             435605
Thefts                                     431659
Vandalism/Criminal Mischief                297890
Theft from Vehicle                         259640
Narcotic / Drug Law Violations             180360
Fraud                                      176292
Burglary Residential                       122241
Aggravated Assault No Firearm              103848
Motor Vehicle Theft                         91499
Robbery No Firearm                          71680
DRIVING UNDER THE INFLUENCE                 66617
Robbery Firearm                             56052
Aggravated Assault Firearm                  48277
Disorderly Conduct                          44115
Weapon Violations                           38339
Burglary Non-Residential                    35676
Other Sex Offenses (Not Commercialized)     21524
Rape                                        17617
Prostitution and Commercialized Vice        15934


In [11]:
data.head()

,dispatch_date,dispatch_time,crime_type,point_x,point_y,DateTime,Year,Month,Day,Hour,Minute,Weekday,IsWeekend,location,street_name
0,2023-10-04,22:58:00,Thefts,-75.232271,39.972757,2023-10-04 22:58:00,2023,10,4,22,58,2,0,1300,Main Toll Plaza Ln 84
1,2023-10-04,22:42:00,DRIVING UNDER THE INFLUENCE,-75.178605,40.030809,2023-10-04 22:42:00,2023,10,4,22,42,2,0,200,FRANKFORD AV & ROBBINS ST
2,2023-10-04,22:25:00,All Other Offenses,-75.226049,39.971074,2023-10-04 22:25:00,2023,10,4,22,25,2,0,600,N 25TH ST & W CAMBRIA ST
3,2023-10-04,22:23:00,All Other Offenses,-75.121491,40.001767,2023-10-04 22:23:00,2023,10,4,22,23,2,0,3400,Rt1 S&b At Wissahickon Ave
4,2023-10-04,22:17:00,Other Assaults,-75.143587,40.044345,2023-10-04 22:17:00,2023,10,4,22,17,2,0,5900,N 3RD ST & W ALLEGHENY AV


# Feature Engineering

Distance of crime to nearest police station

In [12]:
import json

min_distances = json.load(open("data\min_distances.json"))["min_distances"]
data["distance_to_nearest_police_station"] = min_distances
data.head()

,dispatch_date,dispatch_time,crime_type,point_x,point_y,DateTime,Year,Month,Day,Hour,Minute,Weekday,IsWeekend,location,street_name,distance_to_nearest_police_station
0,2023-10-04,22:58:00,Thefts,-75.232271,39.972757,2023-10-04 22:58:00,2023,10,4,22,58,2,0,1300,Main Toll Plaza Ln 84,0.010203
1,2023-10-04,22:42:00,DRIVING UNDER THE INFLUENCE,-75.178605,40.030809,2023-10-04 22:42:00,2023,10,4,22,42,2,0,200,FRANKFORD AV & ROBBINS ST,0.007520
2,2023-10-04,22:25:00,All Other Offenses,-75.226049,39.971074,2023-10-04 22:25:00,2023,10,4,22,25,2,0,600,N 25TH ST & W CAMBRIA ST,0.014742
3,2023-10-04,22:23:00,All Other Offenses,-75.121491,40.001767,2023-10-04 22:23:00,2023,10,4,22,23,2,0,3400,Rt1 S&b At Wissahickon Ave,0.007706
4,2023-10-04,22:17:00,Other Assaults,-75.143587,40.044345,2023-10-04 22:17:00,2023,10,4,22,17,2,0,5900,N 3RD ST & W ALLEGHENY AV,0.000326


In [13]:
weather = pd.read_csv('data/weather.csv')
# Droping the first two rows
weather_cleaned = weather.drop([0, 1]).reset_index(drop=True)

# Setting the third row as the header
weather_cleaned.columns = weather.iloc[1]

# Dropping the now redundant third row
weather_cleaned = weather_cleaned.drop(2).reset_index(drop=True)

# Displaying the first few rows of the cleaned weather dataframe
weather_cleaned.head()

1,time,temperature_2m_max (°F),temperature_2m_min (°F),temperature_2m_mean (°F),precipitation_sum (mm),precipitation_hours (h)
0,2006-01-01,45.5,30.5,36.5,0.00,0.0
1,2006-01-02,46.9,31.1,38.0,12.60,9.0
2,2006-01-04,42.7,31.5,36.3,0.30,1.0
3,2006-01-05,50.5,37.5,42.5,0.00,0.0
4,2006-01-06,41.3,32.6,37.0,0.00,0.0


In [14]:
# Converting the 'time' column in weather_cleaned to datetime format
weather_cleaned['time'] = pd.to_datetime(weather_cleaned['time'])

# Converting the 'dispatch_date' column in data to datetime format
data['dispatch_date'] = pd.to_datetime(data['dispatch_date'])

# Merging the two dataframes based on the date
merged_data = pd.merge(data, weather_cleaned, left_on='dispatch_date', right_on='time', how='left')
data = merged_data

# Displaying the first few rows of the merged dataframe
data.head()

,dispatch_date,dispatch_time,crime_type,point_x,point_y,DateTime,Year,Month,Day,Hour,...,IsWeekend,location,street_name,distance_to_nearest_police_station,time,temperature_2m_max (°F),temperature_2m_min (°F),temperature_2m_mean (°F),precipitation_sum (mm),precipitation_hours (h)
0,2023-10-04,22:58:00,Thefts,-75.232271,39.972757,2023-10-04 22:58:00,2023,10,4,22,...,0,1300,Main Toll Plaza Ln 84,0.010203,2023-10-04,82.0,60.2,69.9,0.00,0.0
1,2023-10-04,22:42:00,DRIVING UNDER THE INFLUENCE,-75.178605,40.030809,2023-10-04 22:42:00,2023,10,4,22,...,0,200,FRANKFORD AV & ROBBINS ST,0.007520,2023-10-04,82.0,60.2,69.9,0.00,0.0
2,2023-10-04,22:25:00,All Other Offenses,-75.226049,39.971074,2023-10-04 22:25:00,2023,10,4,22,...,0,600,N 25TH ST & W CAMBRIA ST,0.014742,2023-10-04,82.0,60.2,69.9,0.00,0.0
3,2023-10-04,22:23:00,All Other Offenses,-75.121491,40.001767,2023-10-04 22:23:00,2023,10,4,22,...,0,3400,Rt1 S&b At Wissahickon Ave,0.007706,2023-10-04,82.0,60.2,69.9,0.00,0.0
4,2023-10-04,22:17:00,Other Assaults,-75.143587,40.044345,2023-10-04 22:17:00,2023,10,4,22,...,0,5900,N 3RD ST & W ALLEGHENY AV,0.000326,2023-10-04,82.0,60.2,69.9,0.00,0.0


In [15]:
unemp = pd.read_csv('data/unemployment rate.csv')
# Converting 'label' column in unemp dataframe to string type
unemp['label'] = unemp['label'].astype(str)

# Extracting month and year from dispatch_date
merged_data['Year-Month'] = merged_data['dispatch_date'].dt.strftime('%Y-%m')

# Merging with unemployment data
merged_data = pd.merge(merged_data, unemp, left_on='Year-Month', right_on='label', how='left')

# If there are any missing values, filling them with the most recent unemployment rate
most_recent_rate = unemp['Unemployment Rate of a Population'].iloc[0]
merged_data['Unemployment Rate of a Population'].fillna(most_recent_rate, inplace=True)

# Dropping unnecessary columns
merged_data.drop(columns=["label", "dispatch_date", "label", "Year-Month"], inplace=True)
merged_data.rename({"Unemployment Rate of a Population": "Unemployment Rate"})
data = merged_data
# Displaying the cleaned dataframe
data.head()

,dispatch_time,crime_type,point_x,point_y,DateTime,Year,Month,Day,Hour,Minute,...,location,street_name,distance_to_nearest_police_station,time,temperature_2m_max (°F),temperature_2m_min (°F),temperature_2m_mean (°F),precipitation_sum (mm),precipitation_hours (h),Unemployment Rate of a Population
0,22:58:00,Thefts,-75.232271,39.972757,2023-10-04 22:58:00,2023,10,4,22,58,...,1300,Main Toll Plaza Ln 84,0.010203,2023-10-04,82.0,60.2,69.9,0.00,0.0,5.4
1,22:42:00,DRIVING UNDER THE INFLUENCE,-75.178605,40.030809,2023-10-04 22:42:00,2023,10,4,22,42,...,200,FRANKFORD AV & ROBBINS ST,0.007520,2023-10-04,82.0,60.2,69.9,0.00,0.0,5.4
2,22:25:00,All Other Offenses,-75.226049,39.971074,2023-10-04 22:25:00,2023,10,4,22,25,...,600,N 25TH ST & W CAMBRIA ST,0.014742,2023-10-04,82.0,60.2,69.9,0.00,0.0,5.4
3,22:23:00,All Other Offenses,-75.121491,40.001767,2023-10-04 22:23:00,2023,10,4,22,23,...,3400,Rt1 S&b At Wissahickon Ave,0.007706,2023-10-04,82.0,60.2,69.9,0.00,0.0,5.4
4,22:17:00,Other Assaults,-75.143587,40.044345,2023-10-04 22:17:00,2023,10,4,22,17,...,5900,N 3RD ST & W ALLEGHENY AV,0.000326,2023-10-04,82.0,60.2,69.9,0.00,0.0,5.4


In [16]:
data.drop(columns=["time"], inplace=True)

Taking care of missing values

In [17]:
data.isna().sum()

dispatch_time                             0
crime_type                                0
point_x                               43047
point_y                               43047
DateTime                                  0
Year                                      0
Month                                     0
Day                                       0
Hour                                      0
Minute                                    0
Weekday                                   0
IsWeekend                                 0
location                                  0
street_name                               0
distance_to_nearest_police_station        0
temperature_2m_max (°F)                 596
temperature_2m_min (°F)                 596
temperature_2m_mean (°F)                596
precipitation_sum (mm)                  596
precipitation_hours (h)                 596
Unemployment Rate of a Population         0
dtype: int64

In [18]:
data.dropna(inplace=True)

In [19]:
data.columns

Index(['dispatch_time', 'crime_type', 'point_x', 'point_y', 'DateTime', 'Year',
       'Month', 'Day', 'Hour', 'Minute', 'Weekday', 'IsWeekend', 'location',
       'street_name', 'distance_to_nearest_police_station',
       'temperature_2m_max (°F)', 'temperature_2m_min (°F)',
       'temperature_2m_mean (°F)', 'precipitation_sum (mm)',
       'precipitation_hours (h)', 'Unemployment Rate of a Population'],
      dtype='object')

In [20]:
data.drop(["dispatch_time", "DateTime"], axis=1, inplace=True)

In [21]:
data.head()

,crime_type,point_x,point_y,Year,Month,Day,Hour,Minute,Weekday,IsWeekend,location,street_name,distance_to_nearest_police_station,temperature_2m_max (°F),temperature_2m_min (°F),temperature_2m_mean (°F),precipitation_sum (mm),precipitation_hours (h),Unemployment Rate of a Population
0,Thefts,-75.232271,39.972757,2023,10,4,22,58,2,0,1300,Main Toll Plaza Ln 84,0.010203,82.0,60.2,69.9,0.00,0.0,5.4
1,DRIVING UNDER THE INFLUENCE,-75.178605,40.030809,2023,10,4,22,42,2,0,200,FRANKFORD AV & ROBBINS ST,0.007520,82.0,60.2,69.9,0.00,0.0,5.4
2,All Other Offenses,-75.226049,39.971074,2023,10,4,22,25,2,0,600,N 25TH ST & W CAMBRIA ST,0.014742,82.0,60.2,69.9,0.00,0.0,5.4
3,All Other Offenses,-75.121491,40.001767,2023,10,4,22,23,2,0,3400,Rt1 S&b At Wissahickon Ave,0.007706,82.0,60.2,69.9,0.00,0.0,5.4
4,Other Assaults,-75.143587,40.044345,2023,10,4,22,17,2,0,5900,N 3RD ST & W ALLEGHENY AV,0.000326,82.0,60.2,69.9,0.00,0.0,5.4


In [22]:
data.to_csv("data\data_full_features.csv")

# Preprocessing Pipeline

* One hot encoding crime types
* Nominally encoding street names
* Standardizing (Z score) different numerical features

In [23]:
data

,crime_type,point_x,point_y,Year,Month,Day,Hour,Minute,Weekday,IsWeekend,location,street_name,distance_to_nearest_police_station,temperature_2m_max (°F),temperature_2m_min (°F),temperature_2m_mean (°F),precipitation_sum (mm),precipitation_hours (h),Unemployment Rate of a Population
0,Thefts,-75.232271,39.972757,2023,10,4,22,58,2,0,1300,Main Toll Plaza Ln 84,0.010203,82.0,60.2,69.9,0.00,0.0,5.4
1,DRIVING UNDER THE INFLUENCE,-75.178605,40.030809,2023,10,4,22,42,2,0,200,FRANKFORD AV & ROBBINS ST,0.007520,82.0,60.2,69.9,0.00,0.0,5.4
2,All Other Offenses,-75.226049,39.971074,2023,10,4,22,25,2,0,600,N 25TH ST & W CAMBRIA ST,0.014742,82.0,60.2,69.9,0.00,0.0,5.4
3,All Other Offenses,-75.121491,40.001767,2023,10,4,22,23,2,0,3400,Rt1 S&b At Wissahickon Ave,0.007706,82.0,60.2,69.9,0.00,0.0,5.4
4,Other Assaults,-75.143587,40.044345,2023,10,4,22,17,2,0,5900,N 3RD ST & W ALLEGHENY AV,0.000326,82.0,60.2,69.9,0.00,0.0,5.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137731,Weapon Violations,-75.232361,39.953284,2006,1,1,0,4,6,1,400,N 18TH ST,0.001530,45.5,30.5,36.5,0.00,0.0,6.7
3137732,Other Assaults,-75.232392,39.963229,2006,1,1,0,2,6,1,5500,S 58TH ST,0.009467,45.5,30.5,36.5,0.00,0.0,6.7
3137733,All Other Offenses,-75.155491,39.942416,2006,1,1,0,2,6,1,0,S 55TH ST,0.015623,45.5,30.5,36.5,0.00,0.0,6.7
3137734,All Other Offenses,-75.155491,39.942416,2006,1,1,0,0,6,1,0,CHERRY ST,0.015623,45.5,30.5,36.5,0.00,0.0,6.7


In [24]:
from sklearn.model_selection import train_test_split

feature_data_pipeline = ColumnTransformer([
    ("ordinal", OrdinalEncoder(), ["street_name"]),
    ('passthrough', 'passthrough', ['IsWeekend']), 
    ("numerical", StandardScaler(), [
        "location", "distance_to_nearest_police_station",
        "temperature_2m_max (°F)", "temperature_2m_min (°F)",
        "temperature_2m_mean (°F)", "precipitation_sum (mm)", 
        "precipitation_hours (h)", "Unemployment Rate of a Population"])
])


#Crime Classification:
X_train, X_test, y_train, y_test = train_test_split(data.drop("crime_type", axis=1), data["crime_type"].reshape(-1, 1), test_size=0.2)
crimetype_encoder = OneHotEncoder()
y_train_encoded = crimetype_encoder.fit_transform(y_train)
y_test_encoded = crimetype_encoder.transform(y_test)
X_train_encoded = feature_data_pipeline.fit_transform(X_train)
X_test_encoded = feature_data_pipeline.transform(X_test)

ValueError: Found unknown categories ['LANCASTER AV  / POWELTON AVE', 'FRANKFORD AVE ET ST', 'BALTIMORE AVE 5300 BALTIMORE AV', 'N 8TH ST 104', 'S 61ST STREET AV', '46TH  /  OSAGE AVE', 'N 44TH ST  / ASPEN ST', 'E CLARKSON ST CLARKSON AV', 'LEITHGOW STREET', '20TH ST  /  SPRUCE', 'N 6500 GIRARD', 'FRANKLIN PL 1', 'S FRAZIER ST  /  ELLIOTT ST', 'N LEITHGOW ST  /  W CUMBERLAND ST', 'LONGSHORE AV  / MARSDEN ST', '22nd & W Diamond St', 'GREENE ST  /  WYNEVA ST', 'WIDENER PL B', 'N BROAD ST 1438', 'OGONTZ AV  /  W CHELTENHAM AVE', 'N PENN ST 5', 'W GUEST ST', '74TH LINDBERGH BLV GH BLVD', 'GEIGER RD 1', 'DEPARTURE A WEST', 'Kendrick St', 'NORTH 18TH ST AV', 'W 65TH AV  & N 20TH ST', 'S 64TH ST  /  EASTWICK AVE', 'MERRI BRK', 'CHELTEN / GERMANTOWN', 'BATH ST  / ORTHODOX ST', 'GERMANTOWN AV & N ALDER ST', 'BROCKLEHURST ST  /  BROUS AV', 'E Ruscomb St', 'CATHEDRAL RD B304', 'OLD BUSTLETON AV 307', 'S 22ND ST  / WASHINGTON AV', '05TH AND CHAMPLOST', 'CONSHOHOCKEN AVE 304', 'E OLNEY AV  /  E CLA

In [ ]:
#Location Regression:
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 9:], data[["point_x", "point_y"]], test_size=0.2)
location_encoder = StandardScaler()
y_train_encoded = location_encoder.fit_transform(y_train)
y_test_encoded = location_encoder.transform(y_test)

In [ ]:
#Time Regression:
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 9:], data[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Weekday']], test_size=0.2)
time_encoder = StandardScaler()
y_train_encoded = time_encoder.fit_transform(y_train)
y_test_encoded = time_encoder.transform(y_test)